In [1]:
from __future__ import print_function

import IPython as ip
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt
import sklearn as skl
import pandas as pd
import math
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from fig_code import visualize_tree, plot_tree_interactive
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
from pandas.stats.api import ols
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/home/avp/anaconda3/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [2]:
df=pd.read_csv('hour.csv')
dataframe=df
dataframe=dataframe.drop(dataframe.columns[[0,3]], axis=1)
#dataframe.head(2)

In [3]:

dataframe['log_count']=dataframe['cnt'].apply(lambda x: math.log(x+1.0))
dataframe=dataframe.set_index('dteday')
#dataframe.head(2)

In [4]:
dataframe['winter']=dataframe.season.apply(lambda x: 1 if x==1 else 0)
dataframe['spring']=dataframe.season.apply(lambda x: 1 if x==2 else 0)
dataframe['summer']=dataframe.season.apply(lambda x: 1 if x==3 else 0)
dataframe['fall']=dataframe.season.apply(lambda x: 1 if x==4 else 0)
#dataframe.head(2)

In [5]:
dataframe['logcasual']=dataframe['casual'].apply(lambda x: math.log(x+1.0))
dataframe['log_registered']=dataframe['registered'].apply(lambda x: math.log(x+1.0))
dataframe['winter_weather']=dataframe.winter*dataframe.weathersit
dataframe['spring_weather']=dataframe.spring*dataframe.weathersit
dataframe['summer_weather']=dataframe.summer*dataframe.weathersit
dataframe['fall_weather']=dataframe.fall*dataframe.weathersit
dataframe['winter_temp']=dataframe.winter*dataframe.atemp
dataframe['sprint_temp']=dataframe.spring*dataframe.atemp
dataframe['summer_temp']=dataframe.summer*dataframe.atemp
dataframe['fall_temp']=dataframe.fall*dataframe.atemp
dataframe.head(2)

,season,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,...,logcasual,log_registered,winter_weather,spring_weather,summer_weather,fall_weather,winter_temp,sprint_temp,summer_temp,fall_temp
dteday,,,,,,,,,,,,,,,,,,,,,
2011-01-01,1,1,0,0,6,0,1,0.24,0.2879,0.81,...,1.386294,2.639057,1,0,0,0,0.2879,0,0,0
2011-01-01,1,1,1,0,6,0,1,0.22,0.2727,0.80,...,2.197225,3.496508,1,0,0,0,0.2727,0,0,0


In [7]:
plt.plot(dataframe['hr'], dataframe['cnt'])
#plt.show()

In [ ]:
dataframe.plot(x='hr', y=['log_count', 'logcasual', 'log_registered'], style='o', kind='bar', 
                title='Hourly bike count for log of total users', xlim=(0.5, 23.5), ylim=(0, 6))
plt.xlabel('Hour')
plt.ylabel('Total number of users')
plt.show()

In [ ]:
dataframe.plot( x='hr', y='log_count', style='o', kind='line', title='Hourly count', xlim=(0.5, 23.5), ylim=(0, 6) )
plt.xlabel('Hour')
plt.ylabel('Number of users')
plt.show()

In [8]:
numerical_features=dataframe[['season','mnth','weekday','holiday', 'workingday','temp','hum','windspeed',
                               'weathersit', 'atemp','winter','spring','summer','fall',
                               'winter_weather','spring_weather','summer_weather','fall_weather','winter_temp','sprint_temp','summer_temp','fall_temp', 'hr']]

In [9]:
train, test = train_test_split(dataframe, test_size=0.15, random_state=0)
print (dataframe.shape)
print (train.shape)
print (test.shape)

(17379, 29)
(14772, 29)
(2607, 29)


In [10]:
seasons=['spring','summer','winter','fall']
seasonweather=['winter_weather','spring_weather','summer_weather','fall_weather']
seasontemp=['winter_temp','sprint_temp','summer_temp','fall_temp']
features=seasons+seasonweather+seasontemp+['holiday','weekday', 'workingday', 'weathersit','temp',
                                      'atemp','hum','windspeed']
print (len(features))
print (features)

20
['spring', 'summer', 'winter', 'fall', 'winter_weather', 'spring_weather', 'summer_weather', 'fall_weather', 'winter_temp', 'sprint_temp', 'summer_temp', 'fall_temp', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed']


In [32]:
# Regression model
def calcCoef(hour,label):                                 # Will calculate coefficients for each bin
    model = ols(y=train[(train.hr==hour)][label],         
               x=train[(train.hr==hour)][features])
    return model.beta
hour=0
casualCoeff=[]                                      # coefficients for the log of no. of casual users
registeredCoeff=[]                                  # same for registered users
totalCoeff=[]

while hour<24:
    casualCoeff.append(calcCoef(hour,'casual'))
    registeredCoeff.append(calcCoef(hour,'registered'))
    totalCoeff.append(calcCoef(hour, 'cnt'))
    hour+=1


In [13]:
#coefficientsC[0]

In [30]:
def calcCount(coef, data):
    res=coef['intercept']
    for i in features:
        res += coef[i]*data[i]
    return round(res)

In [ ]:
#test.columns`

In [16]:
test=test.reset_index()

In [17]:
test['regPredict']=test.apply(lambda x: calcCount(registeredCoeff[x['hr']],x[features]) ,axis=1)

In [18]:
test['casPredict']=test.apply(lambda x: calcCount(casualCoeff[x['hr']],x[features]) ,axis=1)

In [19]:
test['totPredict']=test.apply(lambda x: calcCount(totalCoeff[x['hr']],x[features]) ,axis=1)

In [20]:
test.columns

Index(['dteday', 'season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday',
       'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual',
       'registered', 'cnt', 'log_count', 'winter', 'spring', 'summer', 'fall',
       'logcasual', 'log_registered', 'winter_weather', 'spring_weather', 'summer_weather', 'fall_weather', 'winter_temp', 'sprint_temp', 'summer_temp',
       'fall_temp', 'regPredict', 'casPredict', 'totPredict'],
      dtype='object')

In [ ]:
#test['totAddition']=test.apply(lambda x: x['regPredict'] + x['casPredict'])

In [21]:
results=test[['casual', 'casPredict', 'registered', 'regPredict', 'cnt', 'totPredict']]
results.head()

,casual,casPredict,registered,regPredict,cnt,totPredict
0,4,2,3,8,7,10
1,1,1,4,6,5,7
2,81,27,662,345,743,372
3,45,50,163,91,208,141
4,83,51,250,221,333,272


In [ ]:
res=results.casual-results.casPredict

In [ ]:
results.head(3)

In [ ]:
#results['rms_cas'] = results.apply(lambda x: x['casual'] - x['casPredict'])

In [22]:
results['casual']=results['casual']+0.01
results['registered']=results['registered']+0.01
results['cnt']=results['cnt']+0.01

/home/avp/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/avp/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/avp/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [ ]:
#results['rmse_cas'] = results.apply(lambda x: (np.square(x['casual'] - x['casPredict'])/x['casual']), axis=1)

In [ ]:
#results['accu_cas'] = (1 - results['rmse_cas'])*100
#acc = results['accu_cas']

In [ ]:
results.head(2)

In [ ]:
#((results['casual'] - results['casPredict'])/(results['casual']))

In [ ]:
results.loc[2595]

In [23]:
squared_error_cas = (np.square((results['casual'] - results['casPredict'])))
squared_error_reg = (np.square((results['registered'] - results['regPredict'])))
squared_error_tot = (np.square((results['cnt'] - results['totPredict'])))
#squared_error

In [24]:
divided_by_n_cas = (squared_error_cas/len(results))
divided_by_n_reg = (squared_error_reg/len(results))
divided_by_n_tot = (squared_error_tot/len(results))
#divided_by_n

In [25]:
squared_error_cas = np.round(divided_by_n_cas, 6)
squared_error_reg = np.round(divided_by_n_reg, 6)
squared_error_tot = np.round(divided_by_n_tot, 6)
#squared_error

In [26]:
mean_error_cas = np.mean(squared_error_cas)
mean_error_reg = np.mean(squared_error_reg)
mean_error_tot = np.mean(squared_error_tot)
#mean_error

In [ ]:
rmsd = np.sqrt(mean_error)

In [27]:
from sklearn.metrics import mean_squared_error

rmse_cas = np.sqrt(mean_squared_error(results['casual'], results['casPredict'] ))
rmse_reg = np.sqrt(mean_squared_error(results['registered'], results['regPredict'] ))
rmse_tot = np.sqrt(mean_squared_error(results['cnt'], results['totPredict'] ))

In [28]:
print (rmse_cas)
print (rmse_reg)
print (rmse_tot)

25.2772725308
67.025443247
79.5938122741


In [ ]:
test['regCntPredicted']=test.logRegPredicted.apply(lambda x: round(math.exp(x)-1))
test['casCntPredicted']=test.logCasPredicted.apply(lambda x: round(math.exp(x)-1))
test['totCntPredicted']=test.logTotPredicted.apply(lambda x: round(math.exp(x)-1))
#test['']

In [11]:
# Regression model; backup copy for reference
def calcCoef(hour,label):                                 # Will calculate coefficients for each bin
    return ols(y=train[(train.hr==hour)][label], 
               x=train[(train.hr==hour)][features]).beta
hour=0
casualCoeff=[]                                      # coefficients for the log of no. of casual users
registeredCoeff=[]                                  # same for registered users
totalCoeff=[]

while hour<24:
    casualCoeff.append(calcCoef(hour,'casual'))
    registeredCoeff.append(calcCoef(hour,'registered'))
    totalCoeff.append(calcCoef(hour, 'cnt'))
    hour+=1
